<a href="https://colab.research.google.com/github/waveology/kitchen/blob/main/introducci%C3%B3n_a_netcdf_y_xarray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introducción al NetCDF con xarray

Descarga de ficheros de datos

In [ ]:
# Datos de temperatura a 2 metros - ERA5 - [1987-2020]  -  res=0.25º
# -------------------------------------------------------------------
!wget https://owncloud.aemet.es/index.php/s/Ns1t4FXmZoi5hK4/download -O era5_t2m_1987_2020_europa_025.rar
!unrar e era5_t2m_1987_2020_europa_025.rar && rm -f era5_t2m_1987_2020_europa_025.rar

## Datos de temperatura a 2 metros - ERA5 - [1987-2020]  -  res=0.50º
## -------------------------------------------------------------------
#!wget https://owncloud.aemet.es/index.php/s/Bxjcb2T5pPq1PYP/download -O era5_t2m_1987_2020_europa_05.rar
#!unrar e era5_t2m_1987_2020_europa_05.rar && rm -f era5_t2m_1987_2020_europa_05.rar

# Datos de temperatura en niveles verticales - ERA5 - [2020]  -  res=0.25º
# -----------------------------------------------------------------------
!wget https://owncloud.aemet.es/index.php/s/NoAwwclSdzI1A1g/download -O era5_pl_t_2020_global_025.rar
!unrar e era5_pl_t_2020_global_025.rar && rm -f era5_pl_t_2020_global_025.rar

Importamos las extensiones xarray y Matplotlib (gráficos)

In [ ]:
# Tratamiento de datos multidimensionales
# ------------------------------------------------
import xarray as xr

# Librería gráfica
# -------------------------------
import matplotlib.pyplot as plt

##Lectura de datos
---

En xarray podemos abrir ficheros con la función [open_dataset](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html). El argumento puede ser un fichero local o bien la URL de un servidor remoto OpenDAP. Aquí trabajaremos con un fichero local que contiene 33 años de reanálisis de datos de temperaturas superficial en Europa.

In [ ]:
ds = xr.open_dataset('/content/era5_t2m_1987_2020_europa_025.nc')

In [ ]:
# Esto es un tipo de dato Dataset
# --------------------------------
ds

In [ ]:
# Variables del Dataset
# ----------------------
ds.data_vars

In [ ]:
# Dimensiones del Dataset
# ------------------------
ds.dims

In [ ]:
# Coordenadas del Dataset
# -----------------------
ds.coords

Las variables del Dataset son datos de tipo DataArray y se puede acceder ellos de la misma forma que se accede a las columnas en Pandas

In [ ]:
# Acceso mediante el nombre entre paréntesis
# -------------------------------------------
ds['t2m']

In [ ]:
# Acceso concatenando el nombre con un punto
# -------------------------------------------
ds.t2m

##Selección: *isel, sel y slice*
---

xarray se asienta en Numpy y por eso los DataArray comparten métodos y su trato resulta familiar.

In [ ]:
# Extracción del DataArray de temperaturas del Dataset
# Los datos están en K y los pasamos a ºC
# ------------------------------------------------------
t2m = ds.t2m - 273.15

In [ ]:
# Podemos usar shape para inspeccionar las dimensiones
# ---------------------------------------------------
t2m.shape

En Numpy seleccionaríamos elementos del array de esta manera:

In [ ]:
# Selección al estilo Numpy, indicando valores o rangos de los índices
# ---------------------------------------------------------------------
t2m[0,10:20,15:25]

En xarray disponemos de dos operadores muy cómodos:

*   ***isel*** : selecciona por índices
*   ***sel***  : selecciona por valores

Veamos unos ejemplos seleccionando tiempos (más info [aquí](https://docs.xarray.dev/en/stable/user-guide/time-series.html)):

In [ ]:
# Selección de los datos correspondientes al primer valor de tiempo
# -----------------------------------------------------------------
t2m.isel(time=0)

In [ ]:
# Selección de los datos correspondientes a un valor específico del tiempo
# ------------------------------------------------------------------------
t2m.sel(time='1987-02-20T12:00:00')

In [ ]:
# ¿Y si el valor específico del tiempo no está presente en los datos?
#  Usamos 'method' para especificar que use el tiempo más próximo.
# -------------------------------------------------------------------------------
#t2m.sel(time='1987-02-18T12:00:00')
t2m.sel(time='1987-02-18T12:00:00',method='nearest')

Para especificar un rango en lugar de un valor concreto usamos la función ***slice***:

In [ ]:
# Selección de los datos correspondientes a un periodo de tiempo
# ---------------------------------------------------------------
t2m.sel(time=slice('1987-02-20T12:00:00', '1987-04-16T12:00:00'))

In [ ]:
# Selección de los datos correspondientes a un periodo de tiempo
# especificando solo mes y año
# ---------------------------------------------------------------
t2m.sel(time=slice('1993-07', '1994-02'))

Las funciones ***sel***, ***isel*** y ***slice*** funcionan exactamente igual con el resto de coordenadas. Algunos ejemplos con la latitud y la longitud:

In [ ]:
# Selección de una ventana geográfica:
# norte = 40º sur = 30º / oeste = -30º / este = 0º
# Nota: en ERA5 las latitudes se almanenan invertidas
# --------------------------------------------------------
t2m.sel(latitude=slice(40,30), longitude=slice(-20,0))

In [ ]:
# isel y sel admiten múltiples selectores simultáneos
# Selección de datos en una fecha específica y en una ventana geográfica:
# ---------------------------------------------------------------------------------------
t2m.sel(time='1987-02-20T12:00:00',latitude=slice(40,30), longitude=slice(-20,0))

In [ ]:
# También pueden encadenarse y funcionan igualemente con Datasets
# ---------------------------------------------------------------
ds.isel(time=0).sel(latitude=slice(40,30), longitude=slice(-20,0))

##Gráficos
---

xarray tiene una prestación muy cómoda en relación con los gráficos: intenta deducir a partir de las dimensiones de los datos qué tipo de dibujo queremos.

Por ejemplo si eliminamos la dimensión temporal, la función *plot()* representará un mapa:

In [ ]:
# Seleccionamos un tiempo y una ventana geográfica  f(x,y,t) --> f=f(x,y)
# ------------------------------------------------------------------------
t2m.isel(time=0).sel(latitude = slice(45,36) , longitude = slice(-13,5)).plot(cmap = 'jet', size = 7)

In [ ]:
# Seleccionamos rangos de las tres variables:  f(x,y,t) --> f=f(x,y,t)
# -----------------------------------------------------------------------------------------------------
t2m.sel(time=slice('2001-01','2004-05'), latitude=slice(45,36), longitude=slice(-13,5)).plot(size=5)


In [ ]:
# Seleccionamos un rango de tiempos y las coordenadas de un punto:  f(x,y,t) --> f=f(t)
# ----------------------------------------------------------------------------------------
t2m.sel(time=slice('2001-01','2004-05')).sel(latitude=40.4,longitude=-3.7,method = 'nearest').plot(size=6)

**Sobre la personalización de los gráficos**: la función *plot()* admite varios argumentos que permiten hacer ajustes en los gráficos (más info [aquí](https://docs.xarray.dev/en/stable/user-guide/plotting.html)). Una alternativa sencilla pasa por crear un contenedor y pasar los ejes como argumento a *plot()*:

In [ ]:
# Creamos una figura
# --------------------
fig, ax = plt.subplots()

# Pasamos los ejes como argumento
# ----------------------------------
t2m.sel(time=slice('2001-01','2005-12')).sel(latitude=40.4, longitude=-3.7, method = 'nearest').plot(ax=ax)

# Personalizamos el gráfico añadiendo elementos
# ----------------------------------------------
ax.grid(True)
ax.set_xlabel('Tiempo')
ax.set_ylabel('Temperatura')
ax.set_title('Evolución de la temperatura a 2m en el periodo 2001-2005, en Madrid')

Veamos un ejemplo con un fichero que contiene además una coordenada vertical.

In [ ]:
# Leemos el fichero
# --------------------
ds = xr.open_dataset('/content/era5_pl_t_2020_global_025.nc')

In [ ]:
# Dataset
# ----------
ds

In [ ]:
# Extraemos el DataArray de temperaturas temp = temp(t,x,y,z)
# Pasamos de Kelvin a ºC
# -------------------------------------------------------------
t = ds.t - 273.15

t

In [ ]:
# Extraemos el mapa correspondiente al primer valor de tiempo y al último nivel de presión (1000 hPa)
# Trazamos un paralelo y un meridiano para los cortes verticales
# -----------------------------------------------------------------
fig,ax = plt.subplots()
t.isel(time=0).isel(level=-1).plot(cmap='jet', ax=ax)
ax.axhline(y=-25)
ax.axvline(x=25)

In [ ]:
# Corte vertical por un meridiano que pasa por África
# --------------------------------------------------------
t.isel(time=0).sel(longitude=25,method='nearest').plot(cmap='jet',yincrease=False,size=7)

In [ ]:
# Corte vertical por un paralelo en la latitud de Australia
# ----------------------------------------------------------
t.isel(time=0).sel(longitude=-25,method='nearest').plot(cmap='jet',yincrease=False,size=7)

##Estadística
---

##Agrupamientos
---

##Remuestreo
---

##Máscaras
---

##Múltiples fuentes
---

##Salida en NetCDF
---